In [1]:
import os
import subprocess
import time
import sys

In [2]:
!apt-get update
!apt-get install -y pciutils

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.6 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,201 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,510 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,571 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [22]:
!nohup ollama serve > /dev/null 2>&1 &


In [5]:
!ollama pull gpt-oss:20b # gpt-oss:20b

In [23]:
!ollama run gpt-oss:20b "Write a short poem about AI in logistics."

Thinking...
User wants a short poem about AI in logistics. So produce a short poem. Probably couplet or quatrain. Should be creative. No constraints. I'll produce maybe 8 lines. Let's make it engaging.
...done thinking.

In warehouses humming, AI takes the wheel—  
It maps each pallet’s path through steel and feel.  

Numbers pulse in neon veins, a silent script,  
Predicting demand, so stock never slips.  

Algorithms whisper in the conveyor’s sigh,  
Optimizing routes as clouds glide by.  

From cargo to cloud, a seamless, swift ballet,  
AI in logistics, turning chaos into play.



In [7]:
# =============================================================================
# MAKER FRAMEWORK: TOWER OF HANOI SOLVER (Enhanced)
# =============================================================================
# This notebook implements a multi-agent system to solve the Tower of Hanoi.
# It includes:
# 1. Real-time LLM interaction via Ollama
# 2. Metric tracking (Tokens, Time, Red Flags)
# 3. Consensus Voting Mechanism
# 4. Video Generation of the solution
# 5. Interactive HTML Dashboard generation


import time
import requests
import json
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation, FFMpegWriter
from typing import Dict, Optional, List, Tuple
from dataclasses import dataclass
from tqdm.auto import tqdm
from datetime import datetime
from IPython.display import HTML, display

# =============================================================================
# 1. METRICS LOGGER
# =============================================================================

class MetricsLogger:
    """Comprehensive logging system for tracking all metrics"""

    def __init__(self):
        self.start_time = None
        self.end_time = None
        self.api_calls = []
        self.red_flags = []
        self.voting_rounds = []
        self.step_metrics = []
        self.total_input_tokens = 0
        self.total_output_tokens = 0

    def start_run(self):
        self.start_time = time.time()

    def end_run(self):
        self.end_time = time.time()

    def log_api_call(self, step_idx: int, input_tokens: int, output_tokens: int, response_time: float):
        self.api_calls.append({
            'step': step_idx,
            'input_tokens': input_tokens,
            'output_tokens': output_tokens,
            'response_time': response_time,
            'timestamp': time.time()
        })
        self.total_input_tokens += input_tokens
        self.total_output_tokens += output_tokens

    def log_red_flag(self, step_idx: int, flag_type: str, reason: str):
        self.red_flags.append({
            'step': step_idx,
            'type': flag_type,
            'reason': reason,
            'timestamp': time.time()
        })

    def log_voting_round(self, step_idx: int, candidates: int, winner_votes: int):
        self.voting_rounds.append({
            'step': step_idx,
            'candidates': candidates,
            'winner_votes': winner_votes,
            'timestamp': time.time()
        })

    def log_step_completion(self, step_idx: int, attempts: int, step_time: float, correct: bool):
        self.step_metrics.append({
            'step': step_idx,
            'attempts': attempts,
            'time': step_time,
            'correct': correct,
            'timestamp': time.time()
        })

    def get_total_runtime(self) -> float:
        if self.end_time and self.start_time:
            return self.end_time - self.start_time
        return 0

    def get_summary(self) -> Dict:
        runtime = self.get_total_runtime()
        return {
            'runtime_seconds': runtime,
            'runtime_minutes': runtime / 60,
            'total_steps': len(self.step_metrics),
            'total_api_calls': len(self.api_calls),
            'total_red_flags': len(self.red_flags),
            'total_voting_rounds': len(self.voting_rounds),
            'total_input_tokens': self.total_input_tokens,
            'total_output_tokens': self.total_output_tokens,
            'total_tokens': self.total_input_tokens + self.total_output_tokens,
            'avg_time_per_step': np.mean([s['time'] for s in self.step_metrics]) if self.step_metrics else 0,
            'avg_tokens_per_call': (self.total_input_tokens + self.total_output_tokens) / len(self.api_calls) if self.api_calls else 0,
            'red_flag_rate': len(self.red_flags) / len(self.api_calls) if self.api_calls else 0,
        }

# =============================================================================
# 2. MODEL MANAGER
# =============================================================================

class ModelManager:
    """Model manager with token counting and Ollama connection"""

    def __init__(self, model_name: str, api_url: str = "http://localhost:11434",
                 logger: Optional[MetricsLogger] = None):
        self.model_name = model_name
        self.api_url = api_url
        self.logger = logger
        print(f"🔗 Connecting to Ollama at {api_url}...")

        try:
            resp = requests.get(f"{self.api_url}/api/tags")
            if resp.status_code == 200:
                print(f"✅ Connected to Ollama.")
        except Exception as e:
            print(f"❌ Connection failed: {e}")

    def count_tokens(self, text: str) -> int:
        """Estimate tokens (~4 chars per token)"""
        return len(text) // 4

    def generate_structured(self, system_prompt: str, user_prompt: str,
                          step_idx: int, temperature: float = 0.1,
                          max_tokens: int = 750, json_mode: bool = True) -> str:

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        payload = {
            "model": self.model_name,
            "messages": messages,
            "stream": False,
            "options": {
                "temperature": temperature,
                "num_predict": max_tokens,
                "top_p": 0.9
            }
        }

        if json_mode:
            payload["format"] = "json"

        input_tokens = self.count_tokens(system_prompt + user_prompt)
        start_time = time.time()

        try:
            response = requests.post(f"{self.api_url}/api/chat", json=payload)
            response.raise_for_status()
            content = response.json()['message']['content'].strip()
            output_tokens = self.count_tokens(content)
            response_time = time.time() - start_time

            if self.logger:
                self.logger.log_api_call(step_idx, input_tokens, output_tokens, response_time)

            return content
        except Exception as e:
            print(f"Error: {e}")
            if self.logger:
                self.logger.log_api_call(step_idx, input_tokens, 0, time.time() - start_time)
            return "{}"

# =============================================================================
# 3. CORE LOGIC & PARSING
# =============================================================================

class TowerOfHanoiSolver:
    def __init__(self, num_disks: int):
        self.num_disks = num_disks
        self.moves = []
        self.states = []

    def solve(self):
        initial_state = [list(range(self.num_disks, 0, -1)), [], []]
        self.moves = []
        self.states = [initial_state]
        self._hanoi_recursive(self.num_disks, 0, 2, 1)
        return self.moves, self.states

    def _hanoi_recursive(self, n, source, target, auxiliary):
        if n == 1:
            self._make_move(1, source, target)
        else:
            self._hanoi_recursive(n - 1, source, auxiliary, target)
            self._make_move(n, source, target)
            self._hanoi_recursive(n - 1, auxiliary, target, source)

    def _make_move(self, disk, from_peg, to_peg):
        current_state = [peg[:] for peg in self.states[-1]]
        moved = current_state[from_peg].pop()
        current_state[to_peg].append(disk)
        self.moves.append((disk, from_peg, to_peg))
        self.states.append(current_state)

class ResponseParser:
    def __init__(self, num_disks: int):
        self.num_disks = num_disks

    def parse(self, response: str):
        try:
            # Clean markdown code blocks if present
            clean_response = re.sub(r'```json\s*', '', response)
            clean_response = re.sub(r'```', '', clean_response)

            json_match = re.search(r'\{.*\}', clean_response, re.DOTALL)
            if not json_match:
                return None, None, "No JSON found"

            data = json.loads(json_match.group())
            move = data.get("move")
            next_state = data.get("next_state")

            if not move or len(move) != 3:
                return None, None, "Invalid move format"
            if not next_state or len(next_state) != 3:
                return None, None, "Invalid state format"

            return move, next_state, None
        except Exception as e:
            return None, None, str(e)

    def validate_move_execution(self, current_state, move, claimed_next_state):
        disk, f, t = move

        # Check source
        if not current_state[f]:
            return "Source peg empty"
        if current_state[f][-1] != disk:
            return f"Top disk is {current_state[f][-1]}, not {disk}"

        # Check target
        if current_state[t] and current_state[t][-1] < disk:
            return "Target disk smaller than moving disk"

        # Check state simulation
        sim_state = [p[:] for p in current_state]
        sim_state[f].pop()
        sim_state[t].append(disk)

        # Deep comparison
        if str(sim_state) != str(claimed_next_state):
            return "Next state doesn't match move execution"
        return None

@dataclass
class RedFlagResult:
    is_flagged: bool
    flag_type: Optional[str] = None
    reason: Optional[str] = None

class RedFlagChecker:
    def __init__(self, max_tokens: int, logger: Optional[MetricsLogger] = None):
        self.max_tokens = max_tokens
        self.logger = logger

    def check(self, response, move, next_state, parse_error, step_idx):
        is_long = len(response.split()) > self.max_tokens
        is_error = parse_error is not None

        flagged = is_long or is_error
        flag_type = "length" if is_long else "format" if is_error else None

        if flagged:
            if self.logger:
                self.logger.log_red_flag(step_idx, flag_type, parse_error or "Length exceeded")

        return RedFlagResult(flagged, flag_type, parse_error)

class VotingSystem:
    def __init__(self, k: int = 3, logger: Optional[MetricsLogger] = None):
        self.k = k
        self.logger = logger
        self.step_votes = {}

    def vote(self, step_idx, move, next_state):
        if step_idx not in self.step_votes:
            self.step_votes[step_idx] = {"candidates": {}, "winner": None, "rounds": 0}

        data = self.step_votes[step_idx]
        # Create hashable key
        key = (tuple(move), tuple(tuple(p) for p in next_state))

        if key not in data["candidates"]:
            data["candidates"][key] = {"move": move, "state": next_state, "votes": 0}

        data["candidates"][key]["votes"] += 1
        data["rounds"] += 1

        candidates = sorted(data["candidates"].values(), key=lambda x: x["votes"], reverse=True)
        leader = candidates[0]

        winner_found = False
        if len(candidates) == 1:
            if leader["votes"] >= self.k:
                winner_found = True
        else:
            if leader["votes"] >= candidates[1]["votes"] + self.k:
                winner_found = True

        if winner_found:
            data["winner"] = leader
            if self.logger:
                self.logger.log_voting_round(step_idx, len(candidates), leader["votes"])
            return leader["move"], leader["state"]
        return None

# =============================================================================
# 4. MAKER AGENT SOLVER
# =============================================================================

class MAKERSolver:
    def __init__(self, model, ground_truth, parser, red_flag, voting, config, logger):
        self.model = model
        self.ground_truth = ground_truth
        self.parser = parser
        self.red_flag = red_flag
        self.voting = voting
        self.config = config
        self.logger = logger
        self.solution_moves = []
        self.solution_states = []

    def solve(self):
        num_steps = len(self.ground_truth.moves)
        current_state = self.ground_truth.states[0]
        self.solution_states.append(current_state)
        previous_move = None

        num_disks = self.config['num_disks']

        # Hints to help the LLM stay on track
        if num_disks % 2 == 1:
            disk1_direction = "For ODD disks, disk 1 moves: 0->2->1->0 (repeat)"
            first_move_hint = "First move: disk 1 from peg 0 to peg 2"
        else:
            disk1_direction = "For EVEN disks, disk 1 moves: 0->1->2->0 (repeat)"
            first_move_hint = "First move: disk 1 from peg 0 to peg 1"

        SYSTEM_PROMPT = f"""You are a Tower of Hanoi solver. Find the SINGLE NEXT MOVE.
RULES:
1. Move one disk at a time
2. Never place larger on smaller
3. Three pegs: 0, 1, 2
GOAL: Move all disks from peg 0 to peg 2

STRATEGY:
{disk1_direction}
- If previous move was NOT disk 1: move disk 1
- If previous move WAS disk 1: make only other legal move

{first_move_hint}

OUTPUT (JSON only):
{{
  "reasoning": "brief explanation",
  "move": [disk, from_peg, to_peg],
  "next_state": [[peg0], [peg1], [peg2]]
}}"""

        print(f"🚀 Starting solver for {num_steps} steps...")
        self.logger.start_run()

        for step_idx in tqdm(range(num_steps)):
            step_start = time.time()

            if step_idx == 0:
                user_prompt = f"""Disks: {num_disks}
INITIAL STATE: {current_state}
First move of puzzle. All disks on peg 0.
Find FIRST move."""
            else:
                user_prompt = f"""Disks: {num_disks}
Previous: {previous_move}
Current: {current_state}
Find next move."""

            winner = None
            attempt = 0
            max_attempts = 15 # Safety break

            while winner is None and attempt < max_attempts:
                attempt += 1
                response = self.model.generate_structured(SYSTEM_PROMPT, user_prompt, step_idx)
                move, next_state, err = self.parser.parse(response)

                flag = self.red_flag.check(response, move, next_state, err, step_idx)
                if flag.is_flagged:
                    continue

                logic_err = self.parser.validate_move_execution(current_state, move, next_state)
                if logic_err:
                    self.logger.log_red_flag(step_idx, "logic", logic_err)
                    continue

                winner = self.voting.vote(step_idx, move, next_state)

            if winner is None:
                print(f"❌ Failed to reach consensus at step {step_idx}")
                self.logger.end_run()
                return False, self.logger.get_summary()

            step_time = time.time() - step_start
            correct_move = self.ground_truth.moves[step_idx]

            # Allow disk mismatch in format as long as pegs are right (strictness check)
            is_correct = list(winner[0]) == list(correct_move)

            self.logger.log_step_completion(step_idx, attempt, step_time, is_correct)

            if not is_correct:
                print(f"\n❌ Error at step {step_idx}")
                print(f"   Expected: {correct_move}, Got: {winner[0]}")
                self.logger.end_run()
                return False, self.logger.get_summary()

            self.solution_moves.append(winner[0])
            self.solution_states.append(winner[1])
            current_state = winner[1]
            previous_move = winner[0]

        self.logger.end_run()
        print(f"\n✅ Successfully solved all {num_steps} steps!")
        return True, self.logger.get_summary()

# =============================================================================
# 5. VISUALIZATION
# =============================================================================

class TowerVisualizer:
    def __init__(self, num_disks, fps=10):
        self.num_disks = num_disks
        self.fps = fps
        self.colors = plt.cm.rainbow(np.linspace(0, 1, num_disks))

    def create_video(self, states, moves, filename):
        print("🎬 Rendering video (this may take a moment)...")
        fig, ax = plt.subplots(figsize=(10, 6))

        def update(frame):
            ax.clear()
            ax.axis('off')
            ax.set_ylim(0, self.num_disks + 2)
            ax.set_xlim(-1, 9)
            state = states[frame]

            if frame > 0:
                move = moves[frame - 1]
                ax.set_title(f"Move {frame}: Disk {move[0]} from Peg {move[1]} to Peg {move[2]}",
                           fontsize=14, pad=20)
            else:
                ax.set_title("Initial State", fontsize=14, pad=20)

            # Draw pegs
            for i, x in enumerate([1.5, 4.5, 7.5]):
                ax.add_patch(patches.Rectangle((x-0.1, 0), 0.2, self.num_disks+1, color='gray'))
                ax.text(x, -0.5, f"Peg {i}", ha='center', fontsize=12, fontweight='bold')

            # Draw disks
            for peg_i, peg in enumerate(state):
                for h, disk in enumerate(peg):
                    width = 0.5 + (disk/self.num_disks)*2
                    x = [1.5, 4.5, 7.5][peg_i] - width/2
                    # Use color based on disk size (fixed index)
                    color = self.colors[disk-1]
                    ax.add_patch(patches.Rectangle((x, h), width, 0.9,
                                                   facecolor=color,
                                                   edgecolor='black', linewidth=1.5))
                    ax.text([1.5, 4.5, 7.5][peg_i], h + 0.45, str(disk),
                           ha='center', va='center', fontsize=10, fontweight='bold', color='white')

        anim = FuncAnimation(fig, update, frames=len(states), interval=1000/self.fps)
        writer = FFMpegWriter(fps=self.fps)
        try:
            anim.save(filename, writer=writer)
            plt.close()
            print(f"✅ Video saved: {filename}")
            return True
        except Exception as e:
            print(f"❌ Video generation failed: {e}")
            print("Is ffmpeg installed?")
            plt.close()
            return False

# =============================================================================
# 6. DASHBOARD GENERATOR
# =============================================================================

def generate_dashboard_html(metrics: Dict, config: Dict, video_path: str = None):
    """Generate comprehensive dashboard HTML"""

    dashboard_data = {
        'total_steps': metrics['total_steps'],
        'num_disks': config['num_disks'],
        'total_api_calls': metrics['total_api_calls'],
        'total_red_flags': metrics['total_red_flags'],
        'red_flag_rate': metrics['red_flag_rate'],
        'runtime_minutes': metrics['runtime_minutes'],
        'avg_time_per_step': metrics['avg_time_per_step'],
        'total_input_tokens': metrics['total_input_tokens'],
        'total_output_tokens': metrics['total_output_tokens'],
        'total_tokens': metrics['total_tokens'],
        'avg_tokens_per_call': metrics['avg_tokens_per_call'],
        'video_path': video_path if video_path else ""
    }

    html = f"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MAKER Dashboard</title>
    <style>
        * {{ margin: 0; padding: 0; box-sizing: border-box; }}
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 20px;
        }}
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }}
        .header h1 {{ font-size: 2.5em; margin-bottom: 10px; }}
        .content {{ padding: 30px; }}
        .metrics-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}
        .metric-card {{
            padding: 25px;
            border-radius: 15px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.1);
            transition: transform 0.3s;
        }}
        .metric-card:hover {{ transform: translateY(-5px); }}
        .metric-card.success {{ background: linear-gradient(135deg, #84fab0 0%, #8fd3f4 100%); }}
        .metric-card.info {{ background: linear-gradient(135deg, #a1c4fd 0%, #c2e9fb 100%); }}
        .metric-card.warning {{ background: linear-gradient(135deg, #ffecd2 0%, #fcb69f 100%); }}
        .metric-card.danger {{ background: linear-gradient(135deg, #ff9a9e 0%, #fecfef 100%); }}
        .metric-label {{
            font-size: 0.9em;
            color: #555;
            margin-bottom: 10px;
            font-weight: 600;
            text-transform: uppercase;
        }}
        .metric-value {{
            font-size: 2.5em;
            font-weight: bold;
            color: #333;
        }}
        .metric-subtitle {{
            font-size: 0.85em;
            color: #666;
            margin-top: 5px;
        }}
        .section {{
            background: #f8f9fa;
            padding: 25px;
            border-radius: 15px;
            margin-bottom: 25px;
        }}
        .section h2 {{
            color: #667eea;
            margin-bottom: 20px;
            font-size: 1.8em;
            border-bottom: 3px solid #667eea;
            padding-bottom: 10px;
        }}
        .cost-breakdown {{
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
        }}
        .cost-item {{
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        }}
        .cost-item h3 {{ color: #667eea; margin-bottom: 15px; }}
        .cost-detail {{
            display: flex;
            justify-content: space-between;
            padding: 10px 0;
            border-bottom: 1px solid #e0e0e0;
        }}
        .cost-detail:last-child {{
            border-bottom: none;
            font-weight: bold;
            font-size: 1.1em;
            color: #667eea;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
        }}
        th, td {{
            padding: 15px;
            text-align: left;
            border-bottom: 2px solid #e0e0e0;
        }}
        th {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            font-weight: 600;
        }}
        tr:hover {{ background: #f5f7fa; }}
        .highlight-row {{ background: #e3f2fd !important; }}
        .alert {{
            padding: 15px;
            border-radius: 10px;
            margin: 20px 0;
            border-left: 5px solid;
        }}
        .alert.warning {{
            background: #fff3e0;
            border-color: #ff9800;
            color: #e65100;
        }}
        .video-container {{
            text-align: center;
            margin: 30px 0;
        }}
        .video-container video {{
            max-width: 100%;
            border-radius: 10px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.2);
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🗼 MAKER Framework Dashboard</h1>
            <p>Tower of Hanoi Solver - Live Run Results</p>
            <p style="font-size: 0.9em; margin-top: 10px;" id="timestamp"></p>
        </div>

        <div class="content">
            <!-- Key Metrics -->
            <div class="metrics-grid">
                <div class="metric-card success">
                    <div class="metric-label">Total Steps</div>
                    <div class="metric-value" id="totalSteps">-</div>
                    <div class="metric-subtitle" id="expectedSteps">-</div>
                </div>

                <div class="metric-card info">
                    <div class="metric-label">Total API Calls</div>
                    <div class="metric-value" id="totalCalls">-</div>
                    <div class="metric-subtitle" id="avgCalls">-</div>
                </div>

                <div class="metric-card warning">
                    <div class="metric-label">Red Flags</div>
                    <div class="metric-value" id="redFlags">-</div>
                    <div class="metric-subtitle" id="flagRate">-</div>
                </div>

                <div class="metric-card danger">
                    <div class="metric-label">Runtime</div>
                    <div class="metric-value" id="runtime">-</div>
                    <div class="metric-subtitle" id="avgTime">-</div>
                </div>
            </div>

            <!-- Video Section -->
            <div class="section">
                <h2>🎬 Solution Visualization</h2>
                <div class="video-container" id="videoContainer">
                    <p>Video will be displayed here</p>
                </div>
            </div>

            <!-- Token Usage -->
            <div class="section">
                <h2>📊 Token Usage Breakdown</h2>
                <div class="cost-breakdown">
                    <div class="cost-item">
                        <h3>Token Statistics</h3>
                        <div class="cost-detail">
                            <span>Total Input Tokens:</span>
                            <span id="inputTokens">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Total Output Tokens:</span>
                            <span id="outputTokens">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Grand Total:</span>
                            <span id="totalTokens">-</span>
                        </div>
                    </div>

                    <div class="cost-item">
                        <h3>Per-Call Averages</h3>
                        <div class="cost-detail">
                            <span>Avg Input Tokens:</span>
                            <span id="avgInput">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Avg Output Tokens:</span>
                            <span id="avgOutput">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Avg Total per Call:</span>
                            <span id="avgTotal">-</span>
                        </div>
                    </div>
                </div>
            </div>

            <!-- Cost Estimation -->
            <div class="section">
                <h2>💰 Compute Cost Estimation (T4 GPU on Colab)</h2>
                <div class="cost-breakdown">
                    <div class="cost-item">
                        <h3>Current Run</h3>
                        <div class="cost-detail">
                            <span>Runtime (hours):</span>
                            <span id="runtimeHours">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Compute Units:</span>
                            <span id="computeUnits">-</span>
                        </div>
                        <div class="cost-detail">
                            <span>Estimated Cost:</span>
                            <span id="estimatedCost">-</span>
                        </div>
                    </div>

                    <div class="cost-item">
                        <h3>Cost Breakdown</h3>
                        <div class="cost-detail">
                            <span>T4 GPU Rate:</span>
                            <span>~1.48 units/hour</span>
                        </div>
                        <div class="cost-detail">
                            <span>Colab Pro Rate:</span>
                            <span>$12.41 per ~100 units</span>
                        </div>
                        <div class="cost-detail">
                            <span>Per Unit Cost:</span>
                            <span>~$0.1241</span>
                        </div>
                    </div>
                </div>
            </div>

            <!-- Scaling Predictions -->
            <div class="section">
                <h2>📈 Scaling Predictions</h2>
                <table>
                    <thead>
                        <tr>
                            <th>Disks</th>
                            <th>Steps</th>
                            <th>Est. API Calls</th>
                            <th>Est. Time</th>
                            <th>Compute Units</th>
                            <th>Est. Cost</th>
                        </tr>
                    </thead>
                    <tbody id="scalingTable">
                        <!-- Will be populated by JavaScript -->
                    </tbody>
                </table>

                <div class="alert warning">
                    <strong>⚠️ Note:</strong> Larger disk counts (15-20) require days or weeks of continuous GPU time and substantial compute costs!
                </div>
            </div>
        </div>
    </div>

    <script>
        function populateDashboard(data) {{
            // Set timestamp
            document.getElementById('timestamp').textContent =
                'Run completed: ' + new Date().toLocaleString();

            // Metrics
            document.getElementById('totalSteps').textContent = data.total_steps;
            document.getElementById('expectedSteps').textContent =
                `Expected: ${{Math.pow(2, data.num_disks) - 1}} (2^${{data.num_disks}}-1)`;

            document.getElementById('totalCalls').textContent = data.total_api_calls;
            document.getElementById('avgCalls').textContent =
                `Avg: ${{(data.total_api_calls / data.total_steps).toFixed(1)}} per step`;

            document.getElementById('redFlags').textContent = data.total_red_flags;
            document.getElementById('flagRate').textContent =
                `Rate: ${{(data.red_flag_rate * 100).toFixed(1)}}% of calls`;

            document.getElementById('runtime').textContent = `${{data.runtime_minutes.toFixed(1)}}m`;
            document.getElementById('avgTime').textContent =
                `${{data.avg_time_per_step.toFixed(1)}}s per step`;

            // Tokens
            document.getElementById('inputTokens').textContent = data.total_input_tokens.toLocaleString();
            document.getElementById('outputTokens').textContent = data.total_output_tokens.toLocaleString();
            document.getElementById('totalTokens').textContent = data.total_tokens.toLocaleString();

            document.getElementById('avgInput').textContent =
                Math.round(data.total_input_tokens / data.total_api_calls).toLocaleString();
            document.getElementById('avgOutput').textContent =
                Math.round(data.total_output_tokens / data.total_api_calls).toLocaleString();
            document.getElementById('avgTotal').textContent =
                Math.round(data.avg_tokens_per_call).toLocaleString();

            // Cost
            const runtimeHours = data.runtime_minutes / 60;
            const computeUnits = runtimeHours * 1.48;
            const cost = computeUnits * 0.1241;

            document.getElementById('runtimeHours').textContent = runtimeHours.toFixed(2);
            document.getElementById('computeUnits').textContent = computeUnits.toFixed(2);
            document.getElementById('estimatedCost').textContent = `$${{cost.toFixed(2)}}`;

            // Scaling predictions
            const callsPerStep = data.total_api_calls / data.total_steps;
            const timePerCall = (data.runtime_minutes * 60) / data.total_api_calls;

            const scalingData = [
                {{disks: 5, steps: 31}},
                {{disks: 6, steps: 63}},
                {{disks: 7, steps: 127}},
                {{disks: 8, steps: 255}},
                {{disks: 15, steps: 32767}},
                {{disks: 20, steps: 1048575}}
            ];

            let tableHTML = '';
            scalingData.forEach(row => {{
                const calls = Math.round(row.steps * callsPerStep);
                const timeMinutes = (calls * timePerCall) / 60;
                const timeHours = timeMinutes / 60;
                const units = timeHours * 1.48;
                const rowCost = units * 0.1241;

                const timeStr = timeHours > 24 ?
                    `${{(timeHours / 24).toFixed(0)}} days` :
                    timeHours > 1 ? `${{timeHours.toFixed(0)}} hrs` :
                    `${{timeMinutes.toFixed(0)}} min`;

                // Highlight the row if it matches current config
                const rowClass = (row.disks === data.num_disks) ? 'class="highlight-row"' : '';

                tableHTML += `
                    <tr ${{rowClass}}>
                        <td><strong>${{row.disks}}</strong></td>
                        <td>${{row.steps.toLocaleString()}}</td>
                        <td>${{calls.toLocaleString()}}</td>
                        <td>${{timeStr}}</td>
                        <td>${{units.toFixed(1)}}</td>
                        <td>$${{rowCost.toFixed(2)}}</td>
                    </tr>
                `;
            }});

            document.getElementById('scalingTable').innerHTML = tableHTML;

            // Video
            if (data.video_path) {{
                document.getElementById('videoContainer').innerHTML = `
                    <video width="800" height="480" controls autoplay loop>
                        <source src="${{data.video_path}}" type="video/mp4">
                        Your browser does not support the video tag.
                    </video>
                    <p style="margin-top:10px; color:#666; font-size:0.9em;">
                       Right-click video to save | <a href="${{data.video_path}}" download>Download MP4</a>
                    </p>
                `;
            }} else {{
                document.getElementById('videoContainer').innerHTML = "<p>No video generated</p>";
            }}
        }}

        // Inject Data from Python
        const dashboardData = {json.dumps(dashboard_data)};

        // Init
        populateDashboard(dashboardData);
    </script>
</body>
</html>"""

    return html

#

In [25]:
# 1. Configuration
CONFIG = {
    'num_disks': 5,              # Keep small (4 or 5) for testing
    'model_name': 'gpt-oss:20b', # Must match Ollama pulled model
    'max_attempts': 3,
    'voting_k': 3,
    'max_tokens': 1000
}

print(f"🗼 initializing MAKER for {CONFIG['num_disks']} disks...")

# 2. Initialize Components
logger = MetricsLogger()
model = ModelManager(CONFIG['model_name'], logger=logger)

# 3. Setup Problem
ground_truth = TowerOfHanoiSolver(CONFIG['num_disks'])
ground_truth.solve()
print(f"📝 Optimal solution has {len(ground_truth.moves)} steps")

# 4. Setup Helpers
parser = ResponseParser(CONFIG['num_disks'])
red_flag = RedFlagChecker(CONFIG['max_tokens'], logger)
voting = VotingSystem(CONFIG['voting_k'], logger)

# 5. Initialize Solver
solver = MAKERSolver(
    model=model,
    ground_truth=ground_truth,
    parser=parser,
    red_flag=red_flag,
    voting=voting,
    config=CONFIG,
    logger=logger
)

# 6. Run Solution
print("\n🚀 STARTING SOLVER AGENT...")
success, metrics = solver.solve()




🗼 initializing MAKER for 5 disks...
🔗 Connecting to Ollama at http://localhost:11434...
✅ Connected to Ollama.
📝 Optimal solution has 31 steps

🚀 STARTING SOLVER AGENT...
🚀 Starting solver for 31 steps...


  0%|          | 0/31 [00:00<?, ?it/s]


✅ Successfully solved all 31 steps!


In [30]:
# 7. Generate Video
video_filename = f"hanoi_{CONFIG['num_disks']}_solution.mp4"
visualizer = TowerVisualizer(CONFIG['num_disks'])
visualizer.create_video(solver.solution_states, solver.solution_moves, video_filename)

🎬 Rendering video (this may take a moment)...
✅ Video saved: hanoi_5_solution.mp4


True

In [33]:
if success:
    # 8. Generate Dashboard
    html_content = generate_dashboard_html(metrics, CONFIG, video_filename)

    # Save the file (standard version with relative link)
    html_filename = f"dashboard.html"
    with open(html_filename, "w", encoding='utf-8') as f:
        f.write(html_content)


    # =========================================================================
    # 9. FIX: Display in Notebook with Embedded Video
    # =========================================================================
    import base64
    from moviepy.editor import VideoFileClip, vfx

    clip = VideoFileClip(video_filename)

    # Slow down to half speed
    slow_clip = clip.fx(vfx.speedx, 0.33)  # use vfx.speedx instead of "speedx"

    # Write the slowed video to a new file
    slow_clip.write_videofile(video_filename)
    # 1. Read the video file and encode it to base64
    with open(video_filename, "rb") as video_file:
        video_bytes = video_file.read()
        video_base64 = base64.b64encode(video_bytes).decode('utf-8')

    # 2. Create a Data URI for the video
    video_data_uri = f"data:video/mp4;base64,{video_base64}"

    # 3. Inject the Data URI into the HTML for the notebook display
    # We replace the filename with the actual video data
    notebook_html = html_content.replace(video_filename, video_data_uri)

    # 4. Display links and the embedded dashboard
    display(HTML(f"<h3>Run Complete</h3><p>Open <a href='{html_filename}' target='_blank'>{html_filename}</a> to view full report (downloadable).</p>"))
    display(HTML(notebook_html))



else:
    # Generate partial dashboard even on failure
    html_content = generate_dashboard_html(metrics, CONFIG, None)
    with open("dashboard_failed.html", "w", encoding='utf-8') as f:
        f.write(html_content)


Moviepy - Building video hanoi_5_solution.mp4.
Moviepy - Writing video hanoi_5_solution.mp4



t:  72%|███████▏  | 213/294 [00:01<00:00, 129.58it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file hanoi_5_solution.mp4, 1800000 bytes wanted but 0 bytes read,at frame 73/98, at time 7.30/9.70 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

t:  77%|███████▋  | 227/294 [00:01<00:00, 128.35it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file hanoi_5_solution.mp4, 1800000 bytes wanted but 0 bytes read,at frame 75/98, at time 7.50/9.70 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+

Moviepy - Done !
Moviepy - video ready hanoi_5_solution.mp4


Disks,Steps,Est. API Calls,Est. Time,Compute Units,Est. Cost
